# Download and import libraries

In [4]:
!pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 4.1MB/s 
     |████████████████████████████████| 133kB 27.0MB/s 
     |████████████████████████████████| 81kB 9.0MB/s 
     |████████████████████████████████| 7.5MB 34.9MB/s 
ERROR: botocore 1.20.76 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [5]:
import string
import re
import random
import math
import ast

import pandas as pd

import nltk
from nltk.tokenize import word_tokenize

from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

import torch

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

import tensorflow as tf
from tensorflow.keras import layers


# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')



100%|██████████| 231508/231508 [00:00<00:00, 322351.19B/s]


In [ ]:
nltk.download('stopwords') #download stopwords from nltk
nltk.download('wordnet') # nltk corpus reader


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Read Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Path for the sales dataset (csv format)
DATA_PATH = 'drive/MyDrive/dataset/dataset.pickle'

In [ ]:
dataframe = pd.read_pickle(DATA_PATH) # read dataframe

In [ ]:
dataframe

,crunchbase_ID,home_text,aboutus_text,overview_text,whatwedo_text,company_text,whoweare_text,AI
0,1916,Skip to main content Products GPU accelerated ...,NaN,NaN,NaN,NaN,NaN,1
1,1917,Our AIs Research Company Careers Get in Touch ...,NaN,NaN,NaN,Our AIs Research Company Careers Get in Touch ...,NaN,1
2,1918,Toggle navigation Product Projects Company His...,NaN,NaN,NaN,NaN,NaN,1
3,1919,Brainpeek Solutions Create a seamless online u...,Brainpeek Solutions Create a seamless online u...,NaN,NaN,NaN,NaN,1
4,1920,The Tool Our Languages Services Extract Produc...,The Tool Our Languages Services Extract Produc...,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...
4889,2735,Username or Email L senord Remember me Norsk S...,Username or Email L senord Remember me Norsk S...,NaN,NaN,NaN,NaN,0
4890,5944,Solutions Solution for distributors Covered re...,NaN,NaN,NaN,NaN,NaN,0
4891,5251,BROWSE PRODUCTS Variety Cases Pasta Mac and Ch...,NaN,NaN,NaN,NaN,NaN,0
4892,4225,Pricing Documentation Community Changelog Logi...,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
dataframe.info() #informations about our dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4894 entries, 0 to 4893
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   crunchbase_ID  4894 non-null   object
 1   home_text      4894 non-null   object
 2   aboutus_text   2212 non-null   object
 3   overview_text  66 non-null     object
 4   whatwedo_text  50 non-null     object
 5   company_text   477 non-null    object
 6   whoweare_text  83 non-null     object
 7   AI             4894 non-null   int64 
dtypes: int64(1), object(7)
memory usage: 306.0+ KB


# Data Cleaning

In [ ]:
dataframe = dataframe.fillna('') # replace Nan values by empty string

In [ ]:
df = pd.DataFrame(columns = ['ID','text','AI']) # create new dataframe 
df['ID'] = dataframe['crunchbase_ID']
df['text'] = dataframe['home_text']+ ' ' + dataframe['aboutus_text'] + ' ' + dataframe['overview_text'] + ' ' + dataframe['whatwedo_text'] + ' ' + dataframe['company_text'] + ' ' + dataframe['whoweare_text']
df['AI'] = dataframe ['AI']

# Data Preparation

In [ ]:
stopword = nltk.corpus.stopwords.words('english') # load english stopword
wn = nltk.WordNetLemmatizer() # load Lemmatizer function

stopword.extend(string.punctuation) # add punctuation to stopword list

In [ ]:
def data_preprocessing(text):
    text = re.split("\W+",text.lower())
    text = [word for word in text if word not in stopword] # remove punctuation and stopword
    text = [word for word in text if not word.isdigit()] # remove digit
    text = [wn.lemmatize(word) for word in text] # lemmatize each words
    return text

In [ ]:
df['text'] = df['text'].apply(lambda x: data_preprocessing(x)) # applying data_prepocessing to our dataframe

In [ ]:
df['text'][0] #first text row

['skip',
 'main',
 'content',
 'product',
 'gpu',
 'accelerated',
 'database',
 'spotlyt',
 'visualisation',
 'solution',
 'retail',
 'telecom',
 'finance',
 'logistics',
 'dna',
 'oil',
 'gas',
 'resource',
 'blog',
 'documentation',
 'technical',
 'support',
 'video',
 'white',
 'paper',
 'u',
 'team',
 'partner',
 'customer',
 'contact',
 'world',
 'fastest',
 'advanced',
 'gpu',
 'database',
 'brytlyt',
 'combine',
 'power',
 'gpus',
 'patent',
 'pending',
 'ip',
 'built',
 'postgresql',
 'gain',
 'within',
 'hour',
 'deployed',
 'time',
 'faster',
 'traditional',
 'system',
 'seamless',
 'integration',
 'existing',
 'infrastructure',
 'using',
 'data',
 'connector',
 'feature',
 'rich',
 'easy',
 'use',
 'tool',
 'based',
 'postgresql',
 'add',
 'gpu',
 'cloud',
 'resource',
 'fly',
 'system',
 'scale',
 'need',
 'let',
 'visualisation',
 'tool',
 'slow',
 'spotlyt',
 'brytlyt',
 'super',
 'fast',
 'analytics',
 'workbench',
 'complete',
 'tool',
 'analyst',
 'data',
 'scientist',

In [ ]:
def remove_mincount(words): # Count and return words with single occurence
  liste= []
  for i in range(1,len(df['text'])):
    liste.extend(df['text'][i]) # create list with all words from dataframe
  mincount=[] 
  a = Counter(liste) # count in a dictionnary each occurence
  for cle,valeur in a.items():
    if valeur == 1:
      mincount.append(cle) 
  words = [word for word in words if word not in mincount] # remove from each text all words who appears only one time in total
  return words

In [ ]:
df['text'] = df['text'].apply(lambda x: remove_mincount(x)) # applying remove_mincount to our dataframe

In [ ]:
df.to_csv('drive/MyDrive/dataset/data_clean.csv') # saving our cleaned data

In [ ]:
df = pd.read_csv('drive/MyDrive/dataset/data_clean.csv') # read cleaned data

# Word Embedding

In [ ]:
def tokenise_id__word(word_list):
    word_list = ast.literal_eval(word_list)  # string word_list into list word_list
    text = ' '.join(word for word in word_list) # join all words from list
    tokens = tokenizer.tokenize(text) # tokenise each words 
    tokens_id = tokenizer.convert_tokens_to_ids(tokens) # convert to tokens ID
    return tokens_id

In [ ]:
df['text'] = df['text'].apply(lambda x: tokenise_id__word(x)) # applying tokenise_ID for each word

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (943 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (814 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1675 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (1094 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (913 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length

Some sentence had more than 512 words, whereas BERT tokenizer have a maximum sequence length of 512.

In [ ]:
df.to_csv('drive/MyDrive/dataset/data_tokenise.csv') # saving our data_tokenise dataframe

In [6]:
df = pd.read_csv('drive/MyDrive/dataset/data_tokenise.csv') # read our data_tokenise
df['text'] = df['text'].apply(lambda x:  ast.literal_eval(x)) # transform string list of tokens ID into list of tokens ID

In [7]:
text_with_len = [[text, df['AI'][i], len(text)] # list of lists where each sublist contains tokenized text, the label of the text and the length of the text
                 for i, text in enumerate(df['text'])]


In [8]:
random.shuffle(text_with_len) # to have both Ai and without AI text in the training batches we need to shuffle the text

In [9]:
text_with_len.sort(key=lambda x: x[2]) # we sort the data by the length of the text

In [10]:
sorted_text_labels = [(text_lab[0], text_lab[1]) for text_lab in text_with_len] # remove the length attribute from all the text

In [11]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_text_labels, output_types=(tf.int32, tf.int32)) #convert the sorted dataset into a TensorFlow input dataset shape

In [13]:
BATCH_SIZE = 32 
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ())) # pad our dataset for each batch

The batch size we are going to use is 32 which means that after processing 32 text, the weights of the neural network will be updated

In [14]:
next(iter(batched_dataset)) # print first batch

(<tf.Tensor: shape=(32, 91), dtype=int32, numpy=
 array([[ 4853,  2065,  6444, ...,     0,     0,     0],
        [ 7592,  4012,  7592, ...,     0,     0,     0],
        [ 2131, 10439,  2393, ...,     0,     0,     0],
        ...,
        [ 4456,  5776,  2105, ...,     0,     0,     0],
        [ 2925,  2110,  4435, ...,     0,     0,     0],
        [ 8833,  2378,  3105, ...,  2243, 11089,  4012]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 0, 0, 1], dtype=int32)>)

# Splitting Data

In [ ]:
TOTAL_BATCHES = math.ceil(len(sorted_text_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10   #10% of the data is for test data
batched_dataset.shuffle(TOTAL_BATCHES)

# divide the dataset into test and training sets

test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

# Creating Model

Model consist of three convolutional neural network layers

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

The three convolutional neural network layers are concatenated together and their output is fed to the first densely connected neural network. The second densely connected neural network is used to predict the output AI(presence/absence).

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab) # values for the hyper parameters of our model
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 10


We create an object of the TEXT_MODEL class and pass the hyper paramters values that we defined in the last step to the constructor of the TEXT_MODEL class

In [ ]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
if OUTPUT_CLASSES == 2:   # compiles the model
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

# Fitting Model

In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS) # we use fit method to train our model

Epoch 1/10
138/138 [==============================] - 25s 170ms/step - loss: 0.5627 - accuracy: 0.6802
Epoch 2/10
138/138 [==============================] - 23s 168ms/step - loss: 0.2459 - accuracy: 0.9012
Epoch 3/10
138/138 [==============================] - 22s 156ms/step - loss: 0.0555 - accuracy: 0.9878
Epoch 4/10
138/138 [==============================] - 23s 167ms/step - loss: 0.0123 - accuracy: 0.9989
Epoch 5/10
138/138 [==============================] - 21s 153ms/step - loss: 0.0052 - accuracy: 0.9998
Epoch 6/10
138/138 [==============================] - 23s 165ms/step - loss: 0.0033 - accuracy: 0.9998
Epoch 7/10
138/138 [==============================] - 21s 155ms/step - loss: 0.0043 - accuracy: 0.9998
Epoch 8/10
138/138 [==============================] - 22s 157ms/step - loss: 0.0022 - accuracy: 0.9998
Epoch 9/10
138/138 [==============================] - 21s 154ms/step - loss: 0.0046 - accuracy: 0.9998
Epoch 10/10
138/138 [==============================] - 21s 154ms/step - l

Accuracy of 99.98% on the training set

In [ ]:
results = text_model.evaluate(test_data)
print(results)

15/15 [==============================] - 0s 18ms/step - loss: 0.4603 - accuracy: 0.8604
[0.460312157869339, 0.8604166507720947]


Accuracy of 86.04% on the test set

# Conclusion

We can use BERT Tokenizer to create word embeddings that can be used to perform text classification. We performed AI analysis of website text and achieved an accuracy of 86.04% on the test set.